# Montar o Drive e extrair o Zip com o modelo do paper

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip "/content/drive/MyDrive/UFRPE/6º Período/Redes Neurais/Projeto/projeto_redes_neurais.zip" -d /content/

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/BoundedQueue.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/ArgvContext.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/ForcePublic.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/Condition.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/Closure.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/BlockingQueue.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/include/limonp/Thread.hpp  
  inflating: /content/projeto_redes_neurais/FunASR/runtime/onnxruntime/third_party/jieba/includ

# Ativar a GPU no Colab

In [1]:
import torch

# Verificar se a GPU está disponível
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo: {device}")

Usando o dispositivo: cuda


# Instalar as dependências

In [1]:
# !pip install -r /content/projeto_redes_neurais/requirements.txt
!pip install -r /content/requirements.txt
!pip install jiwer

  Using cached jiwer-3.1.0-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:00


In [2]:
# Bibliotecas padrão
import os
import sys
import re
import math
import argparse

# Bibliotecas de terceiros
import importlib.metadata
from tqdm import tqdm

# PyTor360.tar.gzch e módulos relacionados
import torch
from torch import nn, optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, random_split

# Torchaudio e torchvision
import torchvision
import torchaudio
import torchaudio.transforms as transforms
from torchaudio.transforms import MelSpectrogram

# Avaliação de ASR
import jiwer
import numpy as np


print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("Torchaudio version:", torchaudio.__version__)
print("Jiwer version:", importlib.metadata.version("jiwer"))

Torch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
Torchaudio version: 2.5.1+cu124
Jiwer version: 3.1.0


# Script para pre-processamento da base

1. Baixe a base de teste

In [ ]:
!wget -P /content/data/raw https://www.openslr.org/resources/12/test-clean.tar.gz

--2025-02-22 20:06:33--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/12/test-clean.tar.gz [following]
--2025-02-22 20:06:33--  https://openslr.elda.org/resources/12/test-clean.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘/content/data/raw/test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M   107MB/s    in 3.1s    

2025-02-22 20:06:36 (107 MB/s) - ‘/content/data/raw/test-clean.tar.gz’ saved [346663984/346663984]



In [3]:
# Base de treino maior
!wget -P /content/data/raw https://www.openslr.org/resources/12/train-clean-100.tar.gz

--2025-02-25 01:18:12--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/12/train-clean-100.tar.gz [following]
--2025-02-25 01:18:12--  https://openslr.elda.org/resources/12/train-clean-100.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘/content/data/raw/train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  22.6MB/s    in 4m 38s  

2025-02-25 01:22:51 (21.9 MB/s) - ‘/content/data/raw/train-clean-100.tar.gz’ saved [6387309499/6387309499]



2. Extraia a base dentro da pasta do projeto

In [ ]:
!tar -xvzf /content/data/raw/test-clean.tar.gz -C /content/data/raw/

tar (child): /content/data/raw/test-clean.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [4]:
# Extrair base maior
!tar -xvzf /content/data/raw/train-clean-100.tar.gz -C /content/data/raw/

A saída de streaming foi truncada nas últimas 5000 linhas.
LibriSpeech/train-clean-100/2136/5143/2136-5143-0024.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0012.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0044.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0003.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0010.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0025.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0043.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0004.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0026.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0015.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0032.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0027.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0039.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0020.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0023.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0007.flac
LibriSpeech/t

3. Verificando se a base foi extraida com sucesso

In [5]:
# dataset = torchaudio.datasets.LIBRISPEECH(root="/content/data/raw/", url="test-clean", download=False)
# Base maior
dataset = torchaudio.datasets.LIBRISPEECH(root="/content/data/raw/", url="train-clean-100", download=False)

print(f"Total de amostras: {len(dataset)}\n")

# Exibir informações das primeiras 5 amostras para evitar saída muito grande
for i in range(min(5, len(dataset))):
    print(f"🔹 Amostra {i}:")
    sample = dataset[i]  # Obtém os dados

    waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id = sample

    print(f"  🎙️ Formato do áudio: {waveform.shape}")
    print(f"  🎚️ Taxa de amostragem: {sample_rate}")
    print(f"  📝 Transcrição: {transcript}")
    print(f"  🆔 Speaker ID: {speaker_id}")
    print(f"  📖 Chapter ID: {chapter_id}")
    print(f"  🗣️ Utterance ID: {utterance_id}\n")

print("✅ Exibição concluída.")


Total de amostras: 28539

🔹 Amostra 0:
  🎙️ Formato do áudio: torch.Size([1, 225360])
  🎚️ Taxa de amostragem: 16000
  📝 Transcrição: CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK
  🆔 Speaker ID: 103
  📖 Chapter ID: 1240
  🗣️ Utterance ID: 0

🔹 Amostra 1:
  🎙️ Formato do áudio: torch.Size([1, 255120])
  🎚️ Taxa de amostragem: 16000
  📝 Transcrição: THAT HAD ITS SOURCE AWAY BACK IN THE WOODS OF THE OLD CUTHBERT PLACE IT WAS REPUTED TO BE AN INTRICATE HEADLONG BROOK IN ITS EARLIER COURSE THROUGH THOSE WOODS WITH DARK SECRETS OF POOL AND CASCADE BUT BY THE TIME IT REACHED LYNDE'S HOLLOW IT WAS A QUIET WELL CONDUCTED LITTLE STREAM
  🆔 Speaker ID: 103
  📖 Chapter ID: 1240
  🗣️ Utterance ID: 1

🔹 Amostra 2:
  🎙️ Formato do áudio: torch.Size([1, 223120])
  🎚️ Taxa de amostragem: 16000
  📝 Transcrição: FOR NOT EVEN A BROOK COULD RUN PAST MI

# Pré-processamento da base de dados


In [6]:
# ==== CONFIGURAÇÕES GERAIS ====
# Diretórios
DATA_DIR = "/content/data/raw"
PROCESSED_DIR = "/content/data/processed"
MODELS_DIR = "/content/models"
DATASETS_DIR = "/content/datasets"

# Criar diretórios
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(DATASETS_DIR, exist_ok=True)

# ==== PRÉ-PROCESSAMENTO ====
class LibriSpeechDataset(Dataset):
    def __init__(self, root_dir, transform=None, vocab=None):
        # self.dataset = torchaudio.datasets.LIBRISPEECH(root=root_dir, url="test-clean", download=True)
        self.dataset = torchaudio.datasets.LIBRISPEECH(root=root_dir, url="train-clean-100", download=True)
        self.transform = transform
        self.vocab = vocab

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, _, transcript, _, _, _ = self.dataset[idx]
        if self.transform:
            waveform = self.transform(waveform)
        transcript_indices = [self.vocab.get(c.lower(), 1) for c in transcript]
        return waveform, torch.tensor(transcript_indices, dtype=torch.long)

def build_vocab(dataset):
    chars = set()
    for i in range(len(dataset)):
        _, _, transcript, _, _, _ = dataset[i]
        chars.update(transcript.lower())
    return {'<blank>': 0, '<unk>': 1, **{c: i+2 for i, c in enumerate(sorted(chars))}}

# ==== PRÉ-PROCESSAMENTO COMPLETO ====
def preprocess_data():
    # Verificar se já foi processado
    if not os.path.exists(os.path.join(PROCESSED_DIR, "full_dataset.pt")):
        print("Iniciando pré-processamento...")

        # Criar dataset raw
        # raw_dataset = torchaudio.datasets.LIBRISPEECH(root=DATA_DIR, url="test-clean", download=True)
        raw_dataset = torchaudio.datasets.LIBRISPEECH(root=DATA_DIR, url="train-clean-100", download=True)

        # Criar vocabulário
        vocab = build_vocab(raw_dataset)
        torch.save(vocab, os.path.join(PROCESSED_DIR, "vocab.pt"))
        print("Vocabulário criado!")

        # Processar áudio e textos
        transform = MelSpectrogram(sample_rate=16000, n_mels=128, normalized=True)
        dataset = LibriSpeechDataset(DATA_DIR, transform=transform, vocab=vocab)

        torch.save(dataset, os.path.join(PROCESSED_DIR, "full_dataset.pt"))
        print("Dataset processado salvo!")

        # Dividir e salvar splits
        train_size = int(0.7 * len(dataset))
        val_size = int(0.15 * len(dataset))
        test_size = len(dataset) - train_size - val_size

        splits = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

        torch.save(splits[0], os.path.join(DATASETS_DIR, "train_set.pt"))
        torch.save(splits[1], os.path.join(DATASETS_DIR, "val_set.pt"))
        torch.save(splits[2], os.path.join(DATASETS_DIR, "test_set.pt"))

        print("Pré-processamento concluído!")
    else:
        print("Dataset já processado. Pulando pré-processamento.")

preprocess_data()  # Executar apenas uma vez

Dataset já processado. Pulando pré-processamento.


### Carrega o dataset pré-processado e exibe os primeiros 3 conteúdos

In [11]:
# Carregar o dataset pré-processado
dataset = torch.load("/content/data/processed/full_dataset.pt")

# Verificar uma amostra
sample_idx = 0
waveform, transcript_indices = dataset[sample_idx]

print("🔍 Exemplo de amostra pré-processada:")
print(f"- Formato do espectrograma: {waveform.shape} (Canais, Frequências, Tempo)")
print(f"- Transcrição (índices): {transcript_indices}")
print(f"- Comprimento da transcrição: {len(transcript_indices)}")

# Decodificar a transcrição
vocab = torch.load("/content/data/processed/vocab.pt")
vocab_inv = {v: k for k, v in vocab.items()}

decoded_text = ''.join([vocab_inv[idx.item()] for idx in transcript_indices if idx not in [0, 1]])
print(f"- Transcrição decodificada: '{decoded_text}'")

🔍 Exemplo de amostra pré-processada:
- Formato do espectrograma: torch.Size([1, 128, 1127]) (Canais, Frequências, Tempo)
- Transcrição (índices): tensor([ 6, 11,  4, 19, 23,  8, 21,  2, 18, 17,  8,  2, 16, 12, 22, 22, 24, 22,
         2, 21,  4,  6, 11,  8, 15,  2, 15, 28, 17,  7,  8,  2, 12, 22,  2, 22,
        24, 21, 19, 21, 12, 22,  8,  7,  2, 16, 12, 22, 22, 24, 22,  2, 21,  4,
         6, 11,  8, 15,  2, 15, 28, 17,  7,  8,  2, 15, 12, 25,  8,  7,  2, 13,
        24, 22, 23,  2, 26, 11,  8, 21,  8,  2, 23, 11,  8,  2,  4, 25, 18, 17,
        15,  8,  4,  2, 16,  4, 12, 17,  2, 21, 18,  4,  7,  2,  7, 12, 19, 19,
         8,  7,  2,  7, 18, 26, 17,  2, 12, 17, 23, 18,  2,  4,  2, 15, 12, 23,
        23, 15,  8,  2, 11, 18, 15, 15, 18, 26,  2,  9, 21, 12, 17, 10,  8,  7,
         2, 26, 12, 23, 11,  2,  4, 15,  7,  8, 21, 22,  2,  4, 17,  7,  2, 15,
         4,  7, 12,  8, 22,  2,  8,  4, 21,  7, 21, 18, 19, 22,  2,  4, 17,  7,
         2, 23, 21,  4, 25,  8, 21, 22,  8,  7,  2,  5

<ipython-input-11-437431cfd7a1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load("/content/data/processed/full_dataset.pt")
<ipython-input-11-437431cfd7

# Modelos do Transformer

In [7]:
# Definição do modelo Transformer Baseline
class TransformerASR(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Linear(128, 256)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=256, nhead=8, batch_first=True),
            num_layers=6
        )
        self.fc = nn.Linear(256, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        return x.log_softmax(dim=-1)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Posições (max_len, 1)
        position = torch.arange(max_len).unsqueeze(1)

        # Termo divisor (1, d_model//2)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() *
            (-math.log(10000.0) / d_model)
        ).unsqueeze(0)

        # PE (max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Registrar buffer diretamente
        self.register_buffer('pe', pe.unsqueeze(0))  # Shape: (1, max_len, d_model)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# Adicionando agora o PositionalEncoding para melhores resultados
class TransformerASR_PE(nn.Module):
    def __init__(self, input_size=128, vocab_size=30, dim_model=512, num_heads=8, num_layers=4):
        super().__init__()
        self.embedding = nn.Linear(input_size, dim_model)
        self.pos_encoder = PositionalEncoding(dim_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_model,
            nhead=num_heads,
            dim_feedforward=2048,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(dim_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        return x.log_softmax(dim=-1)

# Adicionando Data Augment para resultados ainda melhores
class TransformerASR_DA(nn.Module):
    def __init__(self, input_size=128, vocab_size=30, dim_model=512, num_heads=8, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_size, dim_model)
        self.pos_encoder = PositionalEncoding(dim_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_model,
            nhead=num_heads,
            dim_feedforward=2048,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(dim_model, vocab_size)

        # Data augmentation: SpecAugment (aplicado no espaço de embedding)
        # OBS.: SpecAugment é geralmente aplicado nos espectrogramas, mas aqui demonstramos sua aplicação no espaço de embedding.
        self.freq_mask = transforms.FrequencyMasking(freq_mask_param=27)
        self.time_mask = transforms.TimeMasking(time_mask_param=100)

    def forward(self, x):
        # x: (batch, time, features)
        x = self.embedding(x)         # -> (batch, time, dim_model)
        x = self.pos_encoder(x)         # Adiciona informações de posição

        # Aplica data augmentation (apenas em treinamento)
        if self.training:
            # Note que os transforms de SpecAugment do torchaudio esperam uma entrada em (batch, time, freq)
            # Aqui, estamos tratando "dim_model" como se fosse a dimensão "freq" do espectrograma.
            x = self.freq_mask(x)
            x = self.time_mask(x)

        # Passa pelo Transformer Encoder
        x = self.transformer(x)
        # Aplica softmax logarítmico sobre a dimensão do vocabulário
        return x.log_softmax(dim=-1)

# Treinamento do modelo

In [ ]:
# Diretórios
DATA_DIR = "/content/data/raw"
PROCESSED_DIR = "/content/data/processed"
MODELS_DIR = "/content/models"
DATASETS_DIR = "/content/datasets"

def collate_fn(batch):
    waveforms, transcripts = zip(*batch)

    # Padding para áudio
    max_time = max(w.shape[-1] for w in waveforms)
    waveforms_padded = torch.stack([
        torch.nn.functional.pad(w, (0, max_time - w.shape[-1]))
        for w in waveforms
    ]).squeeze(1).permute(0, 2, 1)

    # Padding para texto
    transcript_lengths = torch.tensor([len(t) for t in transcripts], dtype=torch.long)
    transcripts_padded = torch.nn.utils.rnn.pad_sequence(transcripts, batch_first=True, padding_value=0)

    return waveforms_padded, transcripts_padded, transcript_lengths

# ================== TREINAMENTO ==================
def train(model, train_loader, val_loader, device, model_name="transformer", lr=3e-4, epochs=50, patience=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    criterion = nn.CTCLoss(blank=0)

    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for waveforms, transcripts, lengths in train_loader:
            waveforms = waveforms.to(device)
            transcripts = transcripts.to(device)
            lengths = lengths.to(device)

            optimizer.zero_grad()
            logits = model(waveforms)  # Shape: (batch, time, vocab_size)

            input_lengths = torch.full((waveforms.size(0),), logits.size(1), dtype=torch.long, device=device)
            loss = criterion(logits.permute(1, 0, 2), transcripts, input_lengths, lengths)

            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for waveforms, transcripts, lengths in val_loader:
                waveforms = waveforms.to(device)
                transcripts = transcripts.to(device)
                lengths = lengths.to(device)

                logits = model(waveforms)
                input_lengths = torch.full((waveforms.size(0),), logits.size(1), dtype=torch.long, device=device)
                val_loss = criterion(logits.permute(1, 0, 2), transcripts, input_lengths, lengths)
                total_val_loss += val_loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(val_loader)
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        print(f"  LR Atual: {optimizer.param_groups[0]['lr']:.2e}\n")

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            epochs_without_improvement = 0
            model_dir = os.path.join("/content/models", model_name)
            os.makedirs(model_dir, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(model_dir, "best_model.pt"))
            print(f"✅ Novo melhor modelo salvo em epoch {epoch+1}")
        else:
            epochs_without_improvement += 1
            print(f"🚨 Sem melhoria por {epochs_without_improvement} época(s)")
            if epochs_without_improvement >= patience:
                print(f"🚨 Early stopping acionado após {patience} épocas sem melhoria.")
                break

    model_dir = os.path.join("/content/models", model_name)
    os.makedirs(model_dir, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(model_dir, "final_model.pt"))
    print(f"Modelo final salvo em {model_dir}/final_model.pt")


# ================== EXECUÇÃO PRINCIPAL ==================
if __name__ == "__main__":
    # Carregar dados pré-processados
    train_set = torch.load(os.path.join(DATASETS_DIR, "train_set.pt"))
    val_set = torch.load(os.path.join(DATASETS_DIR, "val_set.pt"))
    vocab = torch.load(os.path.join(PROCESSED_DIR, "vocab.pt"))

    # Criar DataLoaders
    train_loader = DataLoader(train_set, batch_size=32, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, collate_fn=collate_fn)

    # Inicializar modelo
    model = TransformerASR_DA(
        input_size=128,
        vocab_size=len(vocab),
        dim_model=512,
        num_heads=8,
        num_layers=2
    )

    # Treinar
    train(
        model,
        train_loader,
        val_loader,
        device="cuda",
        model_name="transformer_da_100",
        lr=1e-4,
        epochs=50
    )

<ipython-input-17-faeffdb4b5bb>:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_set = torch.load(os.path.join(DATASETS_DIR, "train_set.pt"))
<ipython-input-17-faeffd

Epoch 1/50
  Train Loss: 3.6461 | Val Loss: 3.4569
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 1
Epoch 2/50
  Train Loss: 3.2149 | Val Loss: 3.2448
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 2
Epoch 3/50
  Train Loss: 3.1104 | Val Loss: 3.3563
  LR Atual: 1.00e-04

🚨 Sem melhoria por 1 época(s)
Epoch 4/50
  Train Loss: 3.0167 | Val Loss: 3.1482
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 4
Epoch 5/50
  Train Loss: 2.9819 | Val Loss: 3.1162
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 5
Epoch 6/50
  Train Loss: 2.9563 | Val Loss: 3.1139
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 6
Epoch 7/50
  Train Loss: 2.9408 | Val Loss: 3.0821
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 7
Epoch 8/50
  Train Loss: 2.9291 | Val Loss: 2.9703
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 8
Epoch 9/50
  Train Loss: 2.8277 | Val Loss: 2.8377
  LR Atual: 1.00e-04

✅ Novo melhor modelo salvo em epoch 9
Epoch 10/

KeyboardInterrupt: 

# Avaliação do conjunto de testes com métrica WER

In [10]:
# ================== CONFIGURAÇÕES ==================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "/content/models/transformer_da_100/best_model.pt"
TEST_SET_PATH = "/content/datasets/test_set.pt"
VOCAB_PATH = "/content/data/processed/vocab.pt"

# ================== FUNÇÕES AUXILIARES ==================
def decode_predictions(logits, vocab):
    """Converte as saídas do modelo em texto decodificado"""
    idx_to_char = {v: k for k, v in vocab.items()}

    # Decodificação greedy
    _, pred_indices = torch.max(logits, dim=-1)  # (batch, time)

    decoded_texts = []
    for pred in pred_indices:
        current_word = []
        prev_char = None

        # Processar a sequência, removendo caracteres repetidos e blanks
        for idx in pred:
            char = idx_to_char[idx.item()]
            if char == '<blank>':
                prev_char = None
                continue
            if char != prev_char:
                current_word.append(char)
            prev_char = char

        decoded_texts.append(''.join(current_word).replace('<unk>', '').strip())

    return decoded_texts

# ================== AVALIAÇÃO ==================
def evaluate_model():
    # Carregar vocabulário
    vocab = torch.load(VOCAB_PATH, weights_only=False)
    #vocab = torch.load(VOCAB_PATH, map_location=torch.device('cpu'))
    vocab_inv = {v: k for k, v in vocab.items()}  # Inverso do vocabulário

    # Carregar modelo
    model = TransformerASR_DA(vocab_size=len(vocab))
    model.load_state_dict(torch.load(MODEL_PATH, weights_only=False))
    #model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
    model.to(DEVICE)
    model.eval()

    # Carregar dataset de teste
    test_set = torch.load(TEST_SET_PATH, weights_only=False)
    #test_set = torch.load(TEST_SET_PATH, map_location=torch.device('cpu'))
    test_loader = DataLoader(test_set, batch_size=16, collate_fn=collate_fn)

    all_references = []
    all_predictions = []

    with torch.no_grad():
        for waveforms, transcripts, _ in test_loader:
            waveforms = waveforms.to(DEVICE)

            # Fazer previsões
            logits = model(waveforms)
            pred_texts = decode_predictions(logits, vocab)

            # Converter índices das transcrições reais para texto
            true_texts = []
            for seq in transcripts:
                text = []
                for idx in seq:
                    if idx.item() in vocab_inv and idx.item() not in [0, 1]:  # Ignorar <blank> e <unk>
                        text.append(vocab_inv[idx.item()])
                true_texts.append(''.join(text))

            all_references.extend(true_texts)
            all_predictions.extend(pred_texts)

    # Calcular WER
    wer_score = jiwer.wer(all_references, all_predictions)
    print(f"WER: {wer_score * 100:.2f}%")

    # Mostrar exemplos
    print("\nExemplos de previsões:")
    for i in range(5):
        print(f"Real: {all_references[i]}")
        print(f"Previsto: {all_predictions[i]}\n")

if __name__ == "__main__":
    evaluate_model()


WER: 99.42%

Exemplos de previsões:
Real: i don't mind i'm glad you're out of it
Previsto: 

Real: top running among the grass or in the wood would pass unperceived the engineer went to the gate of the corral and opened it
Previsto: e  e

Real: there were the poor to be helped the hungry to be fed the naked to be clothed she would gather the orphan children at her knee to teach them the truths of their faith when they were very poor
Previsto: e ae e    a e    e  o

Real: we had all grown so accustomed to advertising signs she said that we failed to notice how thick they were becoming or how bold and overpowering from a few scattered announcements on fence boards
Previsto: ae e    e e  e  oe

Real: the latter's genial nature and dressy appearance pleased him he knew that drouet was only a travelling salesman and not one of many years at that but the firm of bartlett caryoe and company
Previsto: e  e ae e     a e



# Resultados

\begin{array}{|c|c|c|c|c|c|c|}\hline
\text{TransformerASR} & \text{dim_model} & \text{num_heads} & \text{num_layers} & \text{lr} & \text{WER} \\ \hline
\text{Baseline} & 256 & 8 & 6 & 1e-3 & \text{100%} \\ \hline
\text{Positional_Encoding} & 512 & 4 & 4 & 3e-4 & \text{100%} \\ \hline
\text{Data_augmentation} & 512 & 8 & 2 & 1e-4 & \text{100%}\\ \hline
\text{Data_augmentation com train-clean-100} & 512 & 8 & 2 & 1e-4 & \text{99.42%}\\ \hline
\end{array}

**O último teste foi realizado com a base maior "test-clean-100" mais recomendada para o modelo Transformer, porém só conseguimos testar com um modelo treinado durante 13 épocas por falta de capacidade computacional